# Complete Micro-Learning Agent System with Gradio Interface

In [ ]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict, List
from IPython.display import display, Markdown
import gradio as gr
import time
import re
from datetime import datetime, timedelta

load_dotenv(override=True)


In [42]:
# ====== GUARDRAILS CLASS ======
class LearningGuardrails:
    def __init__(self):
        self.user_requests = {}  # Track requests per user/session
        self.blocked_keywords = [
            "hack", "exploit", "illegal", "drugs", "violence", "weapon",
            "bomb", "kill", "harm", "suicide", "self-harm"
        ]
        self.max_requests_per_hour = 5
        self.max_topic_length = 200
        self.min_topic_length = 10
        
    def validate_input(self, topic, num_lessons, user_id=None):
        """Validate user input with multiple guardrails"""
        
        # 1. Basic input validation
        if not topic or not topic.strip():
            return False, "Please enter a topic to learn about!"
            
        topic = topic.strip()
        
        # 2. Length limits
        if len(topic) < self.min_topic_length:
            return False, f"Topic must be at least {self.min_topic_length} characters long!"
            
        if len(topic) > self.max_topic_length:
            return False, f"Topic must be less than {self.max_topic_length} characters!"
        
        # 3. Lesson count validation
        if num_lessons < 1 or num_lessons > 5:
            return False, "Please choose between 1 and 5 lessons to control costs!"
        
        # 4. Content filtering
        topic_lower = topic.lower()
        for keyword in self.blocked_keywords:
            if keyword in topic_lower:
                return False, "This topic is not suitable for educational content!"
        
        # 5. Educational content check
        if not self._is_educational_topic(topic):
            return False, "Please enter a topic suitable for learning and education!"
        
        # 6. Rate limiting (if user_id provided)
        if user_id and not self._check_rate_limit(user_id):
            return False, "You've reached the hourly limit. Please try again later!"
        
        return True, "Valid input!"
    
    def _is_educational_topic(self, topic):
        """Check if topic seems educational"""
        educational_indicators = [
            "learn", "understand", "how to", "basics", "fundamentals", 
            "introduction", "guide", "skills", "management", "development",
            "principles", "strategies", "techniques", "methods", "best practices"
        ]
        
        topic_lower = topic.lower()
        
        # Must be substantial and contain educational language or business/tech terms
        if len(topic.split()) < 3:
            return False
            
        # Check for educational indicators or common business/technical terms
        has_educational_context = any(indicator in topic_lower for indicator in educational_indicators)
        has_business_context = any(term in topic_lower for term in [
            "business", "management", "enterprise", "company", "organization",
            "technology", "software", "digital", "sustainable", "finance",
            "marketing", "sales", "operations", "strategy", "leadership"
        ])
        
        return has_educational_context or has_business_context
    
    def _check_rate_limit(self, user_id):
        """Simple rate limiting per user"""
        now = datetime.now()
        hour_ago = now - timedelta(hours=1)
        
        if user_id not in self.user_requests:
            self.user_requests[user_id] = []
        
        # Clean old requests
        self.user_requests[user_id] = [
            req_time for req_time in self.user_requests[user_id] 
            if req_time > hour_ago
        ]
        
        # Check limit
        if len(self.user_requests[user_id]) >= self.max_requests_per_hour:
            return False
        
        # Record this request
        self.user_requests[user_id].append(now)
        return True

# Initialize guardrails
guardrails = LearningGuardrails()

In [43]:
# ====== PYDANTIC MODELS ======
class LearningSearchItem(BaseModel):
    reason: str = Field(description="Your pedagogical reasoning for why this search is important for learning this topic.")
    query: str = Field(description="The search term to use for gathering educational content.")
    learning_purpose: str = Field(description="What aspect of learning this supports (e.g., 'foundation concepts', 'practical examples', 'common mistakes')")

class LearningSearchPlan(BaseModel):
    searches: List[LearningSearchItem] = Field(description="A list of web searches to perform to gather content for effective micro-learning lessons.")

class KnowledgeCheck(BaseModel):
    question: str = Field(description="A multiple choice question")
    options: List[str] = Field(description="Exactly 4 answer options (A, B, C, D)")
    correct_answer: str = Field(description="The correct answer option (must match one of the options exactly)")
    explanation: str = Field(description="Why this answer is correct and learning reinforcement")

class MicroLesson(BaseModel):
    title: str = Field(description="Engaging title for this micro-lesson")
    learning_objective: str = Field(description="What the learner will achieve after this lesson")
    content: str = Field(description="Main lesson content in markdown (200-400 words)")
    practical_example: str = Field(description="Real-world example or case study")
    knowledge_check: KnowledgeCheck = Field(description="Multiple choice question to test understanding")
    estimated_time: str = Field(description="Estimated time to complete (e.g., '5 minutes')")

class LearningModuleData(BaseModel):
    course_title: str = Field(description="Overall title for the learning module")
    short_summary: str = Field(description="2-3 sentence overview of what learners will gain")
    micro_lessons: List[MicroLesson] = Field(description="List of micro-lessons")
    follow_up_activities: List[str] = Field(description="Suggested next steps or advanced topics")
    total_estimated_time: str = Field(description="Total time for all lessons")


In [44]:
# ====== AGENT DEFINITIONS ======

# Search Agent
SEARCH_INSTRUCTIONS = "You are an educational content gatherer. Given a search term, you search the web for that term and \
produce a concise summary focused on learning and teaching. The summary must be 2-3 paragraphs and less than 300 \
words. Focus on: clear explanations, practical examples, analogies that help understanding, and different approaches \
to teaching this concept. Write for someone who will create micro-lessons, so prioritize content that helps \
learners understand rather than just informational facts. Include actionable insights and real-world applications."

search_agent = Agent(
    name="Educational Content Gatherer",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)


In [45]:
# Planner Agent
HOW_MANY_SEARCHES = 3

PLANNER_INSTRUCTIONS = f"You are a learning experience designer. Given a topic, come up with a set of web searches \
to gather content for creating effective micro-learning lessons. Plan searches that will help build \
a complete learning experience: foundational concepts, practical examples, common misconceptions, \
and real-world applications. Output {HOW_MANY_SEARCHES} strategic searches."

planner_agent = Agent(
    name="Learning Design Planner",
    instructions=PLANNER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=LearningSearchPlan,
)

# Writer Agent (Dynamic creation based on number of lessons)
def get_learning_instructions(num_lessons):
    return (
        f"You are a micro-learning course designer. Given a topic and research content, create engaging "
        f"bite-sized learning modules with knowledge checks. Create exactly {num_lessons} micro-lessons. "
        f"Each lesson should be 200-400 words and include: clear learning objectives, main concept explanation, "
        f"practical examples, and a multiple choice knowledge check question with exactly 4 options (A, B, C, D). "
        f"Make the questions practical and directly test understanding of the lesson content. "
        f"Make content interactive and learner-friendly with actionable takeaways."
    )

def create_writer_agent(num_lessons):
    return Agent(
        name="Micro-Learning Designer",
        instructions=get_learning_instructions(num_lessons),
        model="gpt-4o-mini",
        output_type=LearningModuleData,
    )


In [46]:
# ====== WORKFLOW FUNCTIONS ======

async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: LearningSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: LearningSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input_text = f"Search term: {item.query}\nReason for searching: {item.reason}\nLearning purpose: {item.learning_purpose}"
    result = await Runner.run(search_agent, input_text)
    return result.final_output

async def generate_learning_content(topic, num_lessons):
    """Run the complete learning generation workflow"""
    try:
        print(f"Starting learning module creation for: {topic}")
        
        # Step 1: Plan searches
        search_plan = await plan_searches(topic)
        
        # Step 2: Perform searches  
        search_results = await perform_searches(search_plan)
        
        # Step 3: Create learning module with specified number of lessons
        writer_agent = create_writer_agent(num_lessons)
        input_text = f"Topic: {topic}\nEducational content gathered: {search_results}"
        
        with trace("Learning Module Creation"):
            result = await Runner.run(writer_agent, input_text)
            learning_module = result.final_output
        
        print("Learning module created successfully!")
        return learning_module, None
        
    except Exception as e:
        error_msg = f"Error generating content: {str(e)}"
        print(error_msg)
        return None, error_msg


In [47]:
# ====== CONTENT FORMATTING ======

def format_learning_content_without_checks(learning_module: LearningModuleData):
    """Format the learning module for display WITHOUT knowledge checks"""
    
    content = f"""
# {learning_module.course_title}

**📝 Summary:** {learning_module.short_summary}

**⏱️ Total Time:** {learning_module.total_estimated_time}

---
"""
    
    # Add each lesson WITHOUT knowledge checks
    for i, lesson in enumerate(learning_module.micro_lessons, 1):
        content += f"""
## Lesson {i}: {lesson.title}

**🎯 Learning Objective:** {lesson.learning_objective}

**⏱️ Time:** {lesson.estimated_time}

### Content
{lesson.content}

### 💡 Practical Example
{lesson.practical_example}

---
"""
    
    # Add follow-up activities
    content += "\n## 🚀 Next Steps:\n"
    for activity in learning_module.follow_up_activities:
        content += f"- {activity}\n"
    
    return content

In [50]:
# ====== GRADIO INTERFACE ======

def create_interactive_learning_interface():
    """Create the main learning interface with interactive knowledge checks"""
    
    # Global variable to store learning module
    current_learning_module = None
    quiz_scores = {}
    
    def start_learning(topic, num_lessons, request: gr.Request):
        nonlocal current_learning_module
        
        # Get user identifier
        user_id = request.client.host if request and request.client else None
        
        # Validate input with guardrails
        is_valid, message = guardrails.validate_input(topic, num_lessons, user_id)
        
        if not is_valid:
            return (f"❌ {message}", "", gr.update(visible=False), 
                   *[gr.update(visible=False) for _ in range(20)])  # Hide all quiz components
        
        try:
            # Create new event loop for this request
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            learning_module, error = loop.run_until_complete(
                generate_learning_content(topic, int(num_lessons))
            )
            loop.close()
            
            if error:
                return (f"❌ {error}", "", gr.update(visible=False),
                       *[gr.update(visible=False) for _ in range(20)])
            
            if learning_module:
                current_learning_module = learning_module
                quiz_scores.clear()  # Reset scores
                
                # Format the learning content WITHOUT knowledge checks
                formatted_content = format_learning_content_without_checks(learning_module)
                
                # Create quiz interface components
                quiz_updates = create_quiz_components(learning_module)
                
                return ("✅ Learning module generated successfully!", 
                       formatted_content, 
                       gr.update(visible=True),
                       *quiz_updates)
            else:
                return ("❌ Failed to generate learning content", "", gr.update(visible=False),
                       *[gr.update(visible=False) for _ in range(20)])
                
        except Exception as e:
            return (f"❌ Error: {str(e)}", "", gr.update(visible=False),
                   *[gr.update(visible=False) for _ in range(20)])
    
    def create_quiz_components(learning_module):
        """Create quiz component updates"""
        updates = []
        
        for i, lesson in enumerate(learning_module.micro_lessons):
            # Question and options
            question_text = f"🧠 **Knowledge Check for Lesson {i+1}: {lesson.title}**\n\n{lesson.knowledge_check.question}"
            updates.append(gr.update(value=question_text, visible=True))  # Question
            updates.append(gr.update(choices=lesson.knowledge_check.options, value=None, visible=True))  # Radio
            updates.append(gr.update(visible=True))  # Submit button
            updates.append(gr.update(value="", visible=False))  # Feedback
        
        # Hide unused quiz components (if less than 5 lessons)
        remaining_components = (5 - len(learning_module.micro_lessons)) * 4
        for _ in range(remaining_components):
            updates.append(gr.update(visible=False))
        
        # Show quiz section and score
        updates.append(gr.update(visible=True))  # Quiz section
        updates.append(gr.update(value="📊 **Score: 0/0 (0%)**", visible=True))  # Score display
        
        return updates
    
    def check_answer(lesson_idx, selected_answer):
        """Check if the selected answer is correct"""
        nonlocal current_learning_module, quiz_scores
        
        if not current_learning_module or selected_answer is None:
            return "⚠️ Please select an answer first!", gr.update(visible=False)
        
        lesson = current_learning_module.micro_lessons[lesson_idx]
        correct_answer = lesson.knowledge_check.correct_answer
        explanation = lesson.knowledge_check.explanation
        
        is_correct = selected_answer.strip() == correct_answer.strip()
        quiz_scores[lesson_idx] = is_correct
        
        if is_correct:
            feedback = f"✅ **Correct!** {explanation}"
        else:
            feedback = f"❌ **Incorrect.** The correct answer is: **{correct_answer}**\n\n💡 **Explanation:** {explanation}"
        
        # Update score
        total_answered = len(quiz_scores)
        correct_count = sum(quiz_scores.values())
        percentage = round((correct_count / total_answered) * 100) if total_answered > 0 else 0
        
        score_text = f"📊 **Score: {correct_count}/{total_answered} ({percentage}%)**"
        if total_answered == len(current_learning_module.micro_lessons):
            if percentage >= 80:
                score_text += " 🎉 **Excellent work!**"
            elif percentage >= 60:
                score_text += " 👍 **Good job!**"
            else:
                score_text += " 📚 **Keep studying!**"
        
        return feedback, gr.update(value=score_text)
    
    with gr.Blocks(title="Micro-Learning Generator", theme=gr.themes.Soft()) as app:
        gr.Markdown("# 🎓 Personal Micro-Learning Generator")
        gr.Markdown("Tell me what you'd like to learn and I'll create a personalized learning experience!")
        
        # Usage guidelines - ALWAYS VISIBLE
        gr.Markdown("""
        ### 📋 Usage Guidelines
        - **Educational topics only** - Business, technology, skills, etc.
        - **5 requests per hour limit** - To manage system costs
        - **1-5 lessons maximum** - Quality over quantity
        - **Be specific** - "Sustainable supply chain for SMEs" vs "sustainability"
        """)
        
        # Input section
        with gr.Row():
            topic_input = gr.Textbox(
                label="What would you like to learn about?", 
                placeholder="e.g., Project management fundamentals for small teams",
                lines=2,
                max_lines=3
            )
        
        with gr.Row():
            num_lessons = gr.Number(
                label="How many lessons would you like?",
                value=3,
                minimum=1,
                maximum=5,
                precision=0
            )
        
        generate_btn = gr.Button("Generate My Learning Module", variant="primary")
        
        # Output sections
        status_output = gr.Textbox(label="Status", lines=2)
        
        # Learning content display (initially hidden)
        with gr.Column(visible=False) as learning_content:
            gr.Markdown("## 📚 Your Learning Module")
            content_display = gr.Markdown()
        
        # Interactive quiz section (initially hidden)
        with gr.Column(visible=True) as quiz_section:
            gr.Markdown("## 🧠 Interactive Knowledge Checks")
            
            # Score display
            score_display = gr.Markdown("📊 **Score: 0/0 (0%)**", visible=True)
            
            # Quiz components for up to 5 lessons
            quiz_components = []
            for i in range(5):
                with gr.Group(visible=True) as quiz_group:
                    question_md = gr.Markdown(f"Question {i+1}")
                    answer_radio = gr.Radio(
                        choices=[], 
                        label="Select your answer:",
                        interactive=True
                    )
                    with gr.Row():
                        submit_btn = gr.Button(f"Submit Answer", variant="secondary")
                    feedback_md = gr.Markdown("", visible=True)
                
                quiz_components.extend([question_md, answer_radio, submit_btn, feedback_md])
                
                # Set up the submit button click event
                submit_btn.click(
                    lambda selected, idx=i: check_answer(idx, selected),
                    inputs=[answer_radio],
                    outputs=[feedback_md, score_display]
                ).then(
                    lambda: gr.update(visible=True),
                    outputs=[feedback_md]
                )
        
        # Set up the main generate button
        generate_btn.click(
            start_learning,
            inputs=[topic_input, num_lessons],
            outputs=[status_output, content_display, learning_content] + quiz_components + [quiz_section, score_display]
        )
    
    return app



In [ ]:
app = create_interactive_learning_interface()
app.launch(share=True, debug=True)